In [11]:
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

def team_data (leagues_name, countries, years):
    total_data = []

    for league,country in zip(leagues_name, countries):
        for year in years:
            #result = {}
            url = 'https://www.transfermarkt.com/' + league + '/tabelle/wettbewerb/' + country + '/saison_id/' +str(year)
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            page = requests.get(url, headers=headers)

            soup = BeautifulSoup(page.content, 'html.parser')
            table = soup.find('div', class_ = 'responsive-table')
            table_body = table.find('tbody')
            teams = table_body.find_all('tr')
            for team in teams:
                result = {}
                league_rank = int(team.find('td', class_ = 'rechts hauptlink').text.split(' ')[0])
                #print(league_rank)
                team_name = team.find('td', class_ = 'no-border-links hauptlink').find('a').get('href').split('/')[1]
                #print(team_name)
                team_data = team.find_all('td', class_='zentriert')[1:]
                num_match = int(team_data[0].text)
                num_win = int(team_data[1].text)
                num_draw = int(team_data[2].text)
                num_lose = int(team_data[3].text)
                goals = team_data[4].text.split(':')
                goal_zade = goals[0]
                goal_khorde = goals[1]
                goal_difference = int(team_data[5].text)
                points = int(team_data[6].text)
                result ['year'] = year
                result['team_name'] = team_name
                #print(result)
                result['rank'] = league_rank
                result['num_match'] = num_match
                result['num_win'] = num_win
                result['num_draw'] = num_draw
                result['num_lose'] = num_lose
                result['goal_zade'] = goal_zade
                result['goal_khorde'] = goal_khorde
                result['goal_difference'] = goal_difference
                result['points'] = points
                #print(result)
                total_data.append(result)
        
    return pd.DataFrame(total_data)
leagues_name = ['premier-league', 'laliga', 'bundesliga', 'serie-a', 'ligue-1']
countries = ['GB1', 'ES1', 'L1', 'IT1', 'FR1']

## Insert years into a list
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]   
df = team_data (leagues_name, countries, years)
df.to_csv('team_data_league.csv')